Повторим для тестовой и тренировочной выборки всё то же, что делали при подборе метода моделирования, с учётом того, что используется только выбранный метод (KNN) и найденные параметры для него и Doc2Vec: vector_size = 5, min_count = 2, n_neighbors = 41

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/kontur-home-assignment/test.tsv
/kaggle/input/kontur-home-assignment/train.tsv


In [2]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 273 kB/s            
     |████████████████████████████████| 8.2 MB 1.6 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymorphy2-dicts-ru

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install texthero

     |████████████████████████████████| 24.2 MB 461 kB/s             
     |████████████████████████████████| 1.5 MB 70.3 MB/s            
     |████████████████████████████████| 10.4 MB 38.1 MB/s            
     |████████████████████████████████| 1.0 MB 67.6 MB/s            
     |████████████████████████████████| 184 kB 68.5 MB/s            
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.2
    Uninstalling srsly-2.4.2:
      Successfully uninstalled srsly-2.4.2
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.6
    Uninstalling catalogue-2.0.6:
      Successfully uninstalled catalogue-2.0.6
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.15
    Uninstalling thinc-8.0.15:
      Successfully uninstalled thinc-8.0.15
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.3
    Uninstalling spacy-3.2.3:
      Successfully uninstalled spacy-3.2.3
  Attempting uninstall: nltk
    Found exi

In [5]:
pip install spaCy -U

     |████████████████████████████████| 6.0 MB 287 kB/s            
     |████████████████████████████████| 653 kB 48.8 MB/s            
     |████████████████████████████████| 457 kB 68.7 MB/s            
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: spaCy
    Found existing installation: spacy 2.3.7
    Uninstalling spacy-2.3.7:
      Successfully uninstalled sp

In [6]:
test_data = pd.read_csv('/kaggle/input/kontur-home-assignment/test.tsv', sep = '\t')
train_data = pd.read_csv('/kaggle/input/kontur-home-assignment/train.tsv', sep = '\t')

In [7]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import texthero as hero
from sklearn.neighbors import KNeighborsClassifier

In [8]:
morph = pymorphy2.MorphAnalyzer()
ru_stopwords = stopwords.words('russian')

In [9]:
prediction = test_data['title']
train_data['title_wo_punctuation'] = train_data['title'].str.replace("[^А-ЯЁа-яё0-9 ]", "")
test_data['title_wo_punctuation'] = test_data['title'].str.replace("[^А-ЯЁа-яё0-9 ]", "")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def converter(sentence):
    list = []
    words = sentence.split()
    analyzer = pymorphy2.MorphAnalyzer()
    list = map(lambda x: analyzer.parse(x)[0].normal_form, words)
    return ' '.join(list)
train_data['lemmatized_title'] = train_data['title_wo_punctuation'].apply(converter)
test_data['lemmatized_title'] = test_data['title_wo_punctuation'].apply(converter)

In [11]:
train_data['clean_title'] = train_data['lemmatized_title'].\
                            apply(lambda x: ' '.join([word for word in x.split() if word not in (ru_stopwords)]))
test_data['clean_title'] = test_data['lemmatized_title'].\
                            apply(lambda x: ' '.join([word for word in x.split() if word not in (ru_stopwords)]))

In [12]:
train_data['tokens'] = train_data['clean_title'].apply(word_tokenize)
test_data['tokens'] = test_data['clean_title'].apply(word_tokenize)

In [13]:
corpus = []
for item in train_data['tokens']:
    for token in item:
        corpus.append([token])

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec(vector_size=5, min_count=2, epochs = 20)

In [15]:
words = [TaggedDocument(word, 'tag') for word in corpus]
model.build_vocab(words)
model.train(words, total_examples=model.corpus_count, epochs=model.epochs)

In [16]:
card2vec_train = [model.infer_vector((train_data['tokens'][i])) 
            for i in range(0,len(train_data['tokens']))]
card2vec_test = [model.infer_vector((test_data['tokens'][i])) 
            for i in range(0,len(test_data['tokens']))]

In [17]:
vectors_train= np.array(card2vec_train).tolist()
train_data['card2vec'] = vectors_train
vectors_test= np.array(card2vec_test).tolist()
test_data['card2vec'] = vectors_test

In [18]:
train_data = pd.concat([train_data, pd.DataFrame(train_data['card2vec'].to_list(), \
                                                 columns = ['1_2vec', '2_2vec','3_2vec', '4_2vec', '5_2vec'], \
                                                 index = train_data.index)], axis=1)
test_data = pd.concat([test_data, pd.DataFrame(test_data['card2vec'].to_list(), \
                                                 columns = ['1_2vec', '2_2vec','3_2vec', '4_2vec', '5_2vec'], \
                                                 index = test_data.index)], axis=1)
train_data['words_number'] = [len(train_data['tokens'][i])
                  for i in train_data.index]
test_data['words_number'] = [len(test_data['tokens'][i])
                  for i in test_data.index]
train_data['words_average'] = [sum(map(len, train_data['clean_title'][i].split()))/len(train_data['clean_title'][i].split())
                  for i in train_data.index]
test_data['words_average'] = [sum(map(len, test_data['clean_title'][i].split()))/len(test_data['clean_title'][i].split())
                  for i in test_data.index]

In [19]:
is_fake = train_data['is_fake']
train_data = train_data.drop(columns = ['is_fake'])
test_data = test_data.drop(columns = ['is_fake'])

In [20]:
train_data = train_data.drop(columns = ['title', 'title_wo_punctuation', 'lemmatized_title', 'clean_title', 'tokens', 'card2vec'])
test_data = test_data.drop(columns = ['title', 'title_wo_punctuation', 'lemmatized_title', 'clean_title', 'tokens', 'card2vec'])

In [21]:
knn_model = KNeighborsClassifier(n_neighbors=41)
knn_model.fit(train_data,is_fake)
is_fake_predicted = knn_model.predict(test_data)

In [22]:
prediction = prediction.to_frame()
prediction['is_fake'] = is_fake_predicted.tolist()

In [23]:
prediction.to_csv('prediction.tsv',index=False, sep = '\t')